In [5]:
# TO DO: 
# Add humidity/wind/etc?  If that route is taken, encapsulate into function
    # minima/maxima should be determined by actual acquired temperatures
    
#gmaps.configure??

#Find out why current query from API is pulling data bout the city itself vs the way it was on the assignment

# Sort hotels by reviews; add price range.

In [6]:
import pandas as pd
import numpy as np
import os
import gmaps
import requests
from pprint import pprint

from config import g_key

gmaps.configure(g_key)

In [7]:
weather_df = pd.read_csv(os.path.join("..","weather_database","WeatherPy_Database.csv"), index_col=0)
weather_df

,City,Country,Lat,Long,Max Temp,Pct Humidity,Pct Cloudiness,Wind Speed,Description
0,Plettenberg Bay,ZA,-34.0527,23.3716,65.46,88,19,5.32,few clouds
1,Atuona,PF,-9.8000,-139.0333,77.95,77,11,16.02,few clouds
2,Novo Aripuana,BR,-5.1206,-60.3797,81.16,78,100,2.21,overcast clouds
3,Tasiilaq,GL,65.6145,-37.6368,15.96,55,12,4.34,few clouds
4,Carnarvon,AU,-24.8667,113.6333,75.27,78,0,13.80,clear sky
...,...,...,...,...,...,...,...,...,...
116,Capinota,BO,-17.7167,-66.2333,80.65,29,8,7.00,clear sky
117,Vanimo,PG,-2.6741,141.3028,77.59,91,100,4.54,overcast clouds
118,Bredasdorp,ZA,-34.5322,20.0403,63.72,63,100,11.50,overcast clouds
119,Gornopravdinsk,RU,60.0500,69.9000,1.47,95,100,6.04,overcast clouds


In [8]:
### disallow unreasonable min/max values
min_temp_threshold = 100
max_temp_threshold = 0

#Input strings for repeatability.
min_temp_input_str = f"Please specify a minimum temperature no higher than {min_temp_threshold}: "
not_numeric_str = f"Chosen value is not a number."
threshold_str = f"Chosen value is outside specified threshold."
min_higher_than_max_str = f"Your max temperature is higher than your min temperature."

min_temp = input(min_temp_input_str)

# iterate on input until a valid value is given
while min_temp.isnumeric() == False or float(min_temp) > min_temp_threshold:
    if min_temp.isnumeric() == False:
        print(not_numeric_str)
    elif float(min_temp) > min_temp_threshold:
        print(threshold_str)
    min_temp = input(min_temp_input_str)
    
# Replace max threshold with the min value (if higher)
max_temp_threshold = max(float(min_temp),float(max_temp_threshold))
max_temp_input_str = f"Please specify a maximum temperature no lower than {int(max_temp_threshold)}: "

max_temp = input(max_temp_input_str)

# iterate on input until a valid value is given
while max_temp.isnumeric() == False or float(max_temp) < max_temp_threshold:
    if max_temp.isnumeric() == False:
        print(not_numeric_str)
    elif float(max_temp) < max_temp_threshold:
        print(threshold_str)
    max_temp = input(max_temp_input_str)
    
# Generate data frame from preferred temperatures and remove blanks
vacation_cities_df = weather_df.loc[(weather_df["Max Temp"] >= float(min_temp)) & (weather_df["Max Temp"] <= float(max_temp))].dropna()

print("------------------------")
print("Cities identified based on search criteria:")
vacation_cities_df

Please specify a minimum temperature no higher than 100:  50
Please specify a maximum temperature no lower than 50:  80


------------------------
Cities identified based on search criteria:


,City,Country,Lat,Long,Max Temp,Pct Humidity,Pct Cloudiness,Wind Speed,Description
0,Plettenberg Bay,ZA,-34.0527,23.3716,65.46,88,19,5.32,few clouds
1,Atuona,PF,-9.8000,-139.0333,77.95,77,11,16.02,few clouds
4,Carnarvon,AU,-24.8667,113.6333,75.27,78,0,13.80,clear sky
5,Lompoc,US,34.6391,-120.4579,61.57,73,20,12.66,few clouds
6,Araguaina,BR,-7.1911,-48.2072,72.57,96,100,6.20,light rain
...,...,...,...,...,...,...,...,...,...
113,Marsh Harbour,BS,26.5412,-77.0636,77.67,63,0,0.83,clear sky
114,Atambua,ID,-9.1061,124.8925,73.56,89,90,1.34,overcast clouds
117,Vanimo,PG,-2.6741,141.3028,77.59,91,100,4.54,overcast clouds
118,Bredasdorp,ZA,-34.5322,20.0403,63.72,63,100,11.50,overcast clouds


In [9]:
gmaps_base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

vacation_cities_df["Hotel Name"] = ""

for index, row in vacation_cities_df.iterrows():
    location = f'{vacation_cities_df.loc[index]["Lat"]},{vacation_cities_df.loc[index]["Long"]}'

    params = {'key':g_key,
              'location':location,
              'radius':5000,
              'type':'logding'
            }
    
    try:
        gmaps_request = requests.get(url=gmaps_base_url, params=params)
        gmaps_json = gmaps_request.json()
        
        #results[0] just yields city name.
        hotel_name = gmaps_json['results'][1]['name']
        
        print(f"Identified {hotel_name} in {row['City']}, {row['Country']}")        
        vacation_cities_df.loc[index, "Hotel Name"] = hotel_name
    except KeyError:
        print(f"No hotels found for {row['City']}, {row['Country']}")
        
vacation_cities_df = vacation_cities_df.dropna()
vacation_cities_df

Identified Thanda Vista B&B in Plettenberg Bay, ZA
Identified Banque Socredo in Atuona, PF
Identified Hospitality Carnarvon in Carnarvon, AU
Identified Holiday Inn Express Lompoc, an IHG Hotel in Lompoc, US
Identified Hotel São Vicente in Araguaina, BR
Identified The Halyards Hotel in Port Alfred, ZA
Identified Pousada Porto Praia in Arraial Do Cabo, BR
Identified The Commodore in Mount Gambier, AU
Identified Hotel Hain in Punta Arenas, CL
Identified The Shingles Riverside Cottages in New Norfolk, AU
Identified Gelvandale Stadium in Port Elizabeth, ZA
Identified Southern Sun Waterfront Cape Town in Cape Town, ZA
Identified Mantra on Collins Hobart in Hobart, AU
Identified Courtyard by Marriott Danbury in Bethel, US
Identified Clarion Pointe Sulphur Springs in Sulphur Springs, US
Identified Diego Imóveis in Cidreira, BR
Identified People ThankYou in Rikitea, PF
Identified The Castle in Jamestown, SH
Identified Bluff Homestead - Guesthouse & Campervan Park in Bluff, NZ
Identified Observa

,City,Country,Lat,Long,Max Temp,Pct Humidity,Pct Cloudiness,Wind Speed,Description,Hotel Name
0,Plettenberg Bay,ZA,-34.0527,23.3716,65.46,88,19,5.32,few clouds,Thanda Vista B&B
1,Atuona,PF,-9.8000,-139.0333,77.95,77,11,16.02,few clouds,Banque Socredo
4,Carnarvon,AU,-24.8667,113.6333,75.27,78,0,13.80,clear sky,Hospitality Carnarvon
5,Lompoc,US,34.6391,-120.4579,61.57,73,20,12.66,few clouds,"Holiday Inn Express Lompoc, an IHG Hotel"
6,Araguaina,BR,-7.1911,-48.2072,72.57,96,100,6.20,light rain,Hotel São Vicente
...,...,...,...,...,...,...,...,...,...,...
113,Marsh Harbour,BS,26.5412,-77.0636,77.67,63,0,0.83,clear sky,Abaco Beach Resort
114,Atambua,ID,-9.1061,124.8925,73.56,89,90,1.34,overcast clouds,Pegadaian Betun
117,Vanimo,PG,-2.6741,141.3028,77.59,91,100,4.54,overcast clouds,Vahmoneh Lodge Vanimo
118,Bredasdorp,ZA,-34.5322,20.0403,63.72,63,100,11.50,overcast clouds,Bredasdorp Country Manor


In [10]:
# reset my index before you forget.  Also don't hardcode this csv.
vacation_cities_df.to_csv("WeatherPy_vacation.csv")

In [ ]:
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Today's Weather</dt><dd>{Max Temp} and {Description}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name}
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in vacation_cities_df.iterrows()]

coordinates_df = vacation_cities_df[["Lat","Long"]]
coordinates_df

fig = gmaps.figure(center=(0.0, 0.0), 
                   zoom_level=1.5,
                   layout = {'width': '800px',
                             'height': '500px'})

markers = gmaps.marker_layer(locations=coordinates_df, info_box_content=hotel_info)
fig.add_layer(markers)

## Call the figure
fig